# Capítulo V: Agentes

In [1]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
#Se instancia la herramienta de búsqueda web
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
]

In [4]:
llm=OpenAI(temperature=0)

In [5]:
from langchain.tools.render import render_text_description
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain import hub

In [6]:
#PromptTemplate: hwchase17/react-chat
"""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

'\nAnswer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}\n'

In [7]:
prompt = hub.pull("hwchase17/react-chat")

In [8]:
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [9]:
llm_with_stop = llm.bind(stop=["\nObservation"])

In [10]:
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_stop | ReActSingleInputOutputParser()

In [11]:
from langchain.agents import AgentExecutor

In [12]:
memory = ConversationBufferMemory(memory_key="chat_history")
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [13]:
agent_executor.invoke({"input": "Hola! Mi nombre es Joaquin"})['output']



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
Final Answer: Hola Joaquin, ¡es un placer conocerte! ¿En qué puedo ayudarte?

> Finished chain.


'Hola Joaquin, ¡es un placer conocerte! ¿En qué puedo ayudarte?'

In [14]:
agent_executor.invoke({"input": "¿Recuerdas cómo me llamo?"})['output']



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
Final Answer: Por supuesto, Joaquin. ¡Es un placer conocerte!

> Finished chain.


'Por supuesto, Joaquin. ¡Es un placer conocerte!'

In [15]:
agent_executor.invoke({"input": "¿Cual fue el marcador del último partido de Colo Colo como local?"})['output']



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Current Search
Action Input: último partido de Colo Colo como local{'title': 'Colo-Colo', 'rankings': '3rd in Chilean Primera División', 'thumbnail': 'https://serpapi.com/searches/65134c135f2203626a199f82/images/75b9dfc42bf701752d1de1fab635cd51550418d8a2dffbc1cc9b779ad9aa157f.png', 'game_spotlight': {'league': 'Chilean Primera División', 'stadium': 'Estadio Monumental David Arellano', 'date': 'Sat, Sep 23', 'stage': 'Full-time', 'teams': [{'name': 'Colo-Colo', 'thumbnail': 'https://serpapi.com/searches/65134c135f2203626a199f82/images/75b9dfc42bf701758ee74cd813b7ccf3cc8fd073f896289e93323494b218db922fb2af06b2f9a6268a42598f64143caf.png', 'score': '6'}, {'name': 'Cobresal', 'thumbnail': 'https://serpapi.com/searches/65134c135f2203626a199f82/images/75b9dfc42bf701758ee74cd813b7ccf3cc8fd073f896289e93323494b218db9289a5a6a16df35e6c012b8cacab33b782.png', 'score': '0'}]}} Do I need to use a tool? No
Final Answe

'El último partido de Colo Colo como local fue contra Cobresal, el marcador fue 6-0 a favor de Colo Colo.'

### Ejemplo vanilla con tools personalizadas

referencia: https://github.com/davila7/langchain-101/blob/main/agents/custom_functions_agents.py

In [18]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

load_dotenv()

True

In [10]:
# Tools
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools import BaseTool
from langchain.agents import initialize_agent

In [21]:
turbo_llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo"
)

In [16]:
# Tools personalizadas
def extrae_nombre(name):
    return "El nombre es "+name

def obtiene_tiempo(lugar):
    #llamado a la api wheater
    # POST como parametro el pais
    return "No podemos acceder al clima en "+lugar

nombre_tool = Tool(
    name= "extrae_nombre",
    func=extrae_nombre,
    description="útil cuando queremos saber el nombre de una persona que participa en una conversación, input debería ser el primer nombre"
)

# Obtener el tiempo de un pais
tiempo_tool = Tool(
    name= "tiempo",
    func=obtiene_tiempo,
    description="útil cuando queremos saber el tiempo de un determinado pais, el input debe ser el nombre del pais"
)

# agregamos todos los tools al array
tools = [nombre_tool, tiempo_tool]

In [19]:
#memory
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=3,
    return_messages=True
)

In [22]:
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=turbo_llm,
    verbose=True,
    max_iteration=3,
    early_stop_method="generate",
    memory=memory
)

In [23]:
agent.run("Cual es el nombre de pila de Bernardo Ohiggins?")



> Entering new AgentExecutor chain...
{
    "action": "extrae_nombre",
    "action_input": "Bernardo"
}
Observation: El nombre es Bernardo
Thought:{
    "action": "Final Answer",
    "action_input": "El nombre de pila de Bernardo O'Higgins es Bernardo."
}

> Finished chain.


"El nombre de pila de Bernardo O'Higgins es Bernardo."

In [24]:
agent.run("Me podrías decir el tiempo de Santiago de Chile hoy?")



> Entering new AgentExecutor chain...
{
    "action": "tiempo",
    "action_input": "Santiago de Chile"
}
Observation: No podemos acceder al clima en Santiago de Chile
Thought:{
    "action": "Final Answer",
    "action_input": "Lo siento, no puedo acceder al clima en Santiago de Chile en este momento."
}

> Finished chain.


'Lo siento, no puedo acceder al clima en Santiago de Chile en este momento.'